### Hierarchical Principal Components Analysis

계층적 PCA(Hierarchical PCA)는 대규모 또는 복잡한 데이터 세트를 효과적으로 분석하기 위한 방법이다. 이 접근법은 데이터를 여러 서브셋으로 분할한 후 각 서브셋에 대해 주성분 분석(PCA)을 별도로 수행한다

Subset별로 PCA를 통해 각 서브셋의 주요 패턴과 구조를 추출하며, 이후 이러한 결과를 종합하여 전체 데이터 세트의 특성을 파악한다. 계층적 PCA는 대규모 데이터 세트를 다루는 데 있어 계산상의 부담을 줄이고, 서브셋 간의 다양한 관계나 특성을 더 깊이 이해할 수 있다는 것이 장점이다 이 방법은 복잡한 구조를 가진 데이터 세트를 세분화하여 분석하는 데 특히 유용하며, 금융 분야에서 주로 쓰이고 있다. 그러나 계층적 PCA는 전체 데이터에 대한 일반적인 PCA와 다른 결과를 줄 가능성이 크기 때문에, 분석 결과 해석 시 데이터 분할 방식과 Subset의 특성을 모두 고려해야 한다

계층적 PCA에는 다음과 같은 단계로 이뤄진다

1. 트리 군집화
2. 준 대각화(Quasi-Diagonalization)
3. 재귀적 이분법

이번절에서는 계층적 군집화를 통해 금융 시계열에서 자산의 배분이 어떻게 이루어지는지 보도록 한다

#### 1. 트리 군집화

관측값 $X$의 $T \times N$행렬을 고려해 보자. 행렬은 $T$기간에 대한 $N$개 변수 계열이다. 이 $N$ 열벡터를 군집의 계층 구조로 병합해 배분을 트리 그래프를 따라 하향식으로 흘러내리고자 한다

첫째, 원소가 $\rho = \{\rho_{i,j}\}_{i,j = 1, \dots, N}$인 $N \times N$ 상관관계 행렬을 계산한다. 여기서 $\rho_{i,j} = \rho [X_i, X_j]$다. 거리 척도는 $d : (X_i, X_j) \subset B \rightarrow \mathbb{R} \in [0,1], d_{i,j} = d[X_i, X_j] = \sqrt{\frac{1}{2}(1 - \rho_{i,j})}$로 정의되며, 여기서 $B = [1,\dots, i, \dots, N]$내의 원소의 카티시안 곱이다. 이는 $N \times N$ 거리 행렬 $D = \{d_{i,j}\}_{i,j = 1, \dots, N}$을 계산할 수 있게 해준다. 행렬 $D$는 다음과 같은 성질을 갖는다

$$d[x, y] \geq 0$$
$$d[x,y] = 0 \Leftrightarrow X = Y$$
$$d[x,y] = d[Y,X]$$
$$d[X,Z] \geq d[x,y] + d[Y,Z]$$

따라서, 행렬 $D$는 적절한 거리 공간이라고 할 수 있다

#### 2. 준 대각화

이 단계는 공분산 행렬의 행과 열을 재구성해 가장 큰 값이 대각선에 위치하게 한다. 공분산의 준대각화는 유용한 속성이다

유사한 금융 상품은 함께 배치되고 다른 상품은 서로 떨어뜨리도록 배치한다. 알고리즘은 아래의 코드와 같이 작동하는데, 연결 행렬의 각 행이 두 branch를 하나로 합친다는 것을 알고 있다. 가지 내의 군집들을 그 구성 성분으로 재귀적으로 교체하여, 군집이 더이상 남지 않을 때까지 반복한다. 이런 교체는 군집화의 순서를 유지하게 하는데, 출력은 군집화되지 않은 원소의 정렬된 리스트이다. 아래의 예제를 통해 준 대각화에 대해서 간단히 살펴보자

#### 3. 재귀적 이분법

2를 통해 준대각 행렬을 얻었다. 역분산 배분은 대각 공분산 행렬에 대해 최적이다. 이 사실들로부터 두 가지 활용이 가능하다

1. 상향식 : 연속된 부분 집합의 분산을 역분산 배분의 분산으로 정의한다
2. 하향식 : 인접한 부분 집합 간의 배분을 이들의 총 분산에 역으로 비례하도록 분할한다. 아래의 알고리즘은 하향식 아이디어를 공식화한다